# Db2 (RESTful) Jupyter Notebook Extensions Tutorial

The SQL code tutorials for Db2 rely on a Jupyter notebook extension, commonly refer to as a "magic" command. The beginning of all of the notebooks begin with the following command which will load the extension and allow the remainder of the notebook to use the `%sql` magic command.
<pre>
&#37;run db2re.ipynb
</pre>

This code defines a Jupyter/Python magic command called `%sql` which allows you to execute Db2 specific calls to 
the database. There are other packages available for manipulating databases, but this one has been specifically
designed for demonstrating a number of the SQL features available in Db2 using RESTful calls. Note that this code will **only** work for a Db2 on Cloud instance and is not supported for connecting to a standard Db2 server. To use this code on with a Db2 server run the **`db2.ipynb`** command instead.

There are two ways of executing the `%sql` command. A single line SQL statement would use the
line format of the magic command:
<pre>
%sql SELECT * FROM EMPLOYEE
</pre>

If you have a large block of sql then you would place the `%%sql` command at the beginning of the block and then
place the SQL statements into the remainder of the block. Using this form of the `%%sql` statement means that the
notebook cell can only contain SQL and no other statements.
<pre>
%%sql
SELECT * FROM EMPLOYEE
ORDER BY LASTNAME
</pre>

You can have multiple lines in the SQL block (`%%sql`). The default SQL delimiter is the semi-column (`;`).
If you have scripts (triggers, procedures, functions) that use the semi-colon as part of the script, you 
will need to use the `-d` option to change the delimiter to an at @ sign. 
<pre>
%%sql -d
SELECT * FROM EMPLOYEE
@
CREATE PROCEDURE ...
@
</pre>

The `%sql` command allows most DB2 commands to execute and has a special version of the CONNECT statement. 

The CONNECT command has the following format:
<pre>
%sql CONNECT CREDENTIALS <variable>
</pre>

The `CREDENTIALS` can be supplied as a variable in which case the contents will be used to create an access token to the database and if successful, the variable will be saved to disk for future use. If you create another notebook and use the identical syntax, if the variable is not defined, the contents on disk will be used as the credentials. You should assign the credentials to a variable that represents the database (or schema) that you are communicating with. Using familiar names makes it easier to remember the credentials when connecting. 

For example, if the credentials are assigned to a variable called sample:
```python
sample = \
{
... credentials ...
}
```
Then the connect statement would be:
```
%sql connect credentials sample
```
If you open up a new notebook, you will not have to create the sample variable with the credentials in it. The `%sql connect` command will determine that the variable does not exist and will read the contents of the file with the same name `sample.pickle` and retrieve the credentials that way.

In addition to the -d option, there are a number different options that you can specify at the beginning of 
the SQL. Some of these options can only be used for single line commands (**`%sql`**) while others can be used for both forms of the command. 

### Options for **`%sql`** commands only

   - `-r` - Return the result set as an array of values instead of a dataframe
   - `-sampledata` - Create and load the EMPLOYEE and DEPARTMENT sample tables
   
The `-sampledata` flag will create the sample tables and not process any SQL statements that may be in your `%sql` block. The `-r` flag is meant for returning an answer set back to a variable. For intance:

```python
results = %sql -r select * from employee
```

You can only run SQL statements within an `%%sql` block, so the statement above would not work. If you have a large SQL statement that you want to assign to a variable, you can use one of two approaches. The first option is to place the contents of your SQL statement into a variable:

```python
somesql = """
SELECT * 
FROM EMPLOYEE
"""
```

Then you can issue the `%sql` call using the form:

```python
results = %sql -r {somesql}
```

The Python interpreter will take the contents of the variable `somesql` and place it into the command. The other option is to use Python line continuation characters. To extend the command beyond one line, place a `\` at the end of each line:

```python
results = %sql -r \
SELECT * \
FROM EMPLOYEE 
```
   
### Options for **`%%sql`** and **`%sql`** commands

   - `-d` - Delimiter: Change SQL delimiter to "`@`" from "`;`"
   - `-q` - Quiet: Quiet results - no messages returned from the function
   - `-a` - All: Display all rows in answer set and do not limit the rows displayed
   - `-e` - Any macro expansions are displayed in an output box 
   
### Options for **`%%sql`** and **`%sql`** Commands that Terminate Blocks

These options will work with both forms of the `%sql` magic command, but will only execute on one SQL statement. What this means is that in a `%%sql` block, only the first SQL statement will execute and have the flag applied to it. In addition, any SQL in a `%%sql` block that generates output will terminate the block.

   - `-j` - JSON: Create a pretty JSON representation. Only the first column is formatted
   - `-pb` - Plot Bar: Plot the results as a bar chart
   - `-pl` - Plot Line: Plot the results as a line chart
   - `-pp` - Plot Pie: Plot the results as a pie chart
   - `-i`  - Interactive plot mode (Only available if you install Pixiedust)


<p>
You can pass python variables to the `%sql` or `%%sql` command by using a colon `:` before the name of the variable. Note that you will need to place proper punctuation around the variable in the event the
SQL command requires it. For instance, the following example will find employee '000010' in the EMPLOYEE table.
<pre>
empno = '000010'
%sql SELECT LASTNAME FROM EMPLOYEE WHERE EMPNO=':empno'
</pre>

Before we run any SQL commands against Db2, we must load the `db2re.ipynb` library.

In [ ]:
%run db2re.ipynb

## Connecting to Db2

Before any SQL commands can be issued, a connection needs to be made to the Db2 database that you will be using.

The CONNECT command has the following format:
```
%sql CONNECT CREDENTIALS <variable>
%sql CONNECT AUTOCONNECT ON | OFF
```

The CREDENTIALS can be supplied as a variable in which case the contents will be used to create an access token to the database. If the connection is successful, the contents of the variable will be saved to disk for future use. If you create another notebook you can use the identical syntax but you will not have to define the variable beforehand - it will be loaded from disk. 

You should assign the credentials to a variable that represents the database (or schema) that you are communicating with. Using familiar names makes it easier to remember the credentials when connecting. For example, if the credentials are assigned to a variable called sample:

```python
sample = \
{
... credentials ...
}
``` 

Then the connect statement would be:

```
%sql connect credentials sample
```


By default, the program will generate an access token for every SQL statement issued (AUTOCONNECT ON). This will generate additional overhead because one extra RESTful call is required before issuing an SQL statement. However, this guarantees that you will always be able to issue the SQL statement. If you turn AUTOCONNECT OFF, the program will generate an access token when you connect, but will never generate an access token after that point. Since tokens expire after a period of time, your SQL calls may stop working. The error code will indicate an authentication error, so you will need to reissue the CONNECT CREDENTIALS statement again to allow the statement to execute.

Enter your Db2 Credentials in the cell below before issuing the `CONNECT` statement. If you have connected in a previous notebook then the credentials will have been saved with the same name as the variable you supplied.

In [ ]:
db2id = \


Request a connection token to the database and turn `AUTOCONNECT OFF` to reduce overhead. Note that if you come back to this notebook at a later time, you may have to re-connect due to connection tokens expiring.

In [ ]:
%sql connect credentials db2id

## Line versus Cell Command
The Db2 extension is made up of one magic command that works either at the LINE level (`%sql`) or at the CELL level (`%%sql`). If you only want to execute a SQL command on one line in your script, use the `%sql` form of the command. If you want to run a larger block of SQL, then use the `%%sql` form. Note that when you use the `%%sql` form of the command, the entire contents of the cell is considered part of the command, so you cannot mix other commands in the cell.

The following is an example of a line command:

In [ ]:
%sql VALUES 'HELLO THERE'

If you have SQL that requires multiple lines, of if you need to execute many lines of SQL, then you should 
be using the CELL version of the `%sql` command. To start a block of SQL, start the cell with `%%sql` and do not place any SQL following the command. Subsequent lines can contain SQL code, with each SQL statement delimited with the semicolon (`;`). You can change the delimiter if required for procedures, etc... More details on this later.

In [ ]:
%%sql
VALUES
  1,
  2,
  3

If you are using a single statement then there is no need to use a delimiter. However, if you are combining a number of commands then you must use the semicolon.

In [ ]:
%%sql
DROP TABLE STUFF;
CREATE TABLE STUFF (A INT);
INSERT INTO STUFF VALUES
  1,2,3;
SELECT * FROM STUFF;

The script will generate messages and output as it executes. Each SQL statement that generates results will have a table displayed with the result set. If a command is executed, the results of the execution get listed as well. The script you just ran probably generated an error on the DROP table command.

## Options
Both forms of the `%sql` command have options that can be used to change the behavior of the code. For both forms of the command (`%sql`, `%%sql`), the options must be on the same line as the command:
<pre>
%sql -t ...
%%sql -t
</pre>

The only difference is that the `%sql` command can have SQL following the parameters, while the `%%sql` requires the SQL to be placed on subsequent lines.

There are a number of parameters that you can specify as part of the `%sql` statement. 

* `-d` - Use alternative delimiter
* `-q` - Suppress messages 
* `-j` - JSON formatting of a column
* `-a` - Show all output
* `-pb` - Bar chart of results
* `-pp` - Pie chart of results 
* `-pl` - Line chart of results
* `-i` - Interactive mode with Pixiedust
* `-sampledata` Load the database with the sample EMPLOYEE and DEPARTMENT tables
* `-r` - Return the results into a variable (list of rows)
* `-e` - Echo macro substitution

Multiple parameters are allowed on a command line. Each option should be separated by a space:
<pre>
%sql -a -j ...
</pre>

A `SELECT` statement will return the results as a dataframe and display the results as a table in the notebook. If you use the assignment statement, the dataframe will be placed into the variable and the results will not be displayed:
<pre>
r = %sql SELECT * FROM EMPLOYEE
</pre>

The sections below will explain the options in more detail.

## Delimiters
The default delimiter for all SQL statements is the semicolon. However, this becomes a problem when you try to create a trigger, function, or procedure that uses SQLPL (or PL/SQL). Use the `-d` option to turn the SQL delimiter into the at (`@`) sign and `-q` to suppress error messages. The semi-colon is then ignored as a delimiter.

For example, the following SQL will use the `@` sign as the delimiter.

In [ ]:
%%sql -d -q
DROP TABLE STUFF
@
CREATE TABLE STUFF (A INT)
@
INSERT INTO STUFF VALUES
  1,2,3
@
SELECT * FROM STUFF
@

The delimiter change will only take place for the statements following the `%%sql` command. Subsequent cells
in the notebook will still use the semicolon. You must use the `-d` option for every cell that needs to use the
semicolon in the script.

## Limiting Result Sets
The default number of rows displayed for any result set is 10. You have the option of changing this option when initially connecting to the database. If you want to override the number of rows display you can either update
the control variable, or use the -a option. The `-a` option will display all of the rows in the answer set. For instance, the following SQL will only show 10 rows even though we inserted 15 values:

In [ ]:
%sql values 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15

You will notice that the displayed result will split the visible rows to the first 5 rows and the last 5 rows.
Using the `-a` option will display all values in a scrollable table.

In [ ]:
%sql -a values 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15

A special note regarding the output from a `SELECT` statement. If the SQL statement is the last line of a block, the results will be displayed by default (unless you assigned the results to a variable). If the SQL is in the middle of a block of statements, the results will displayed and execution past that point will stop.


In [ ]:
%%sql
values 'First Line';
values 'Second Line';

## Quiet Mode
Every SQL statement will result in some output. You will either get an answer set (`SELECT`), or an indication if
the command worked. For instance, the following set of SQL will generate some error messages since the tables 
will probably not exist:

In [ ]:
%%sql
DROP TABLE TABLE_NOT_FOUND;
DROP TABLE TABLE_SPELLED_WRONG;

If you know that these errors may occur you can silence them with the -q option.

In [ ]:
%%sql -q
DROP TABLE TABLE_NOT_FOUND;
DROP TABLE TABLE_SPELLED_WRONG;

SQL output will not be suppressed, so the following command will still show the results.

In [ ]:
%%sql -q
DROP TABLE TABLE_NOT_FOUND;
DROP TABLE TABLE_SPELLED_WRONG;
VALUES 1,2,3;

## Variables in %sql Blocks

The `%sql` syntax allows you to pass local variables to a script. You can pass python variables to the `%sql` or `%%sql` command by using a colon `:` before the name of the variable. Note that you will need to place proper punctuation around the variable in the event the SQL command requires it. For instance, the following example will find employee '000010' in the EMPLOYEE table.
```
empno = '000010'
%sql SELECT LASTNAME FROM EMPLOYEE WHERE EMPNO=':empno'
```

In [ ]:
empno = '000010'
%sql SELECT LASTNAME FROM EMPLOYEE WHERE EMPNO=':empno'

## JSON Formatting
Db2 supports querying JSON that is stored in a column within a table. Standard output would just display the 
JSON as a string. For instance, the following statement would just return a large string of output.

In [ ]:
%%sql
VALUES 
      '{
      "empno":"000010",
      "firstnme":"CHRISTINE",
      "midinit":"I",
      "lastname":"HAAS",
      "workdept":"A00",
      "phoneno":[3978],
      "hiredate":"01/01/1995",
      "job":"PRES",
      "edlevel":18,
      "sex":"F",
      "birthdate":"08/24/1963",
      "pay" : {
        "salary":152750.00,
        "bonus":1000.00,
        "comm":4220.00}
      }'

Adding the -j option to the `%sql` (or `%%sql`) command will format the first column of a return set to better
display the structure of the document. Note that if your answer set has additional columns associated with it, they will not be displayed in this format.

In [ ]:
%%sql -j
VALUES 
      '{
      "empno":"000010",
      "firstnme":"CHRISTINE",
      "midinit":"I",
      "lastname":"HAAS",
      "workdept":"A00",
      "phoneno":[3978],
      "hiredate":"01/01/1995",
      "job":"PRES",
      "edlevel":18,
      "sex":"F",
      "birthdate":"08/24/1963",
      "pay" : {
        "salary":152750.00,
        "bonus":1000.00,
        "comm":4220.00}
      }'

JSON fields can be inserted into Db2 columns using Python dictionaries. This makes the input and output of JSON fields much simpler. For instance, the following code will create a Python dictionary which is similar to a JSON record.

In [ ]:
employee = {
    "firstname" : "John",
    "lastname" : "Williams",
    "age" : 45
}

The field can be inserted into a character column (or BSON if you use the JSON functions) by doing a direct variable insert.

In [ ]:
%%sql -q
DROP TABLE SHOWJSON;
CREATE TABLE SHOWJSON(JSONIN VARCHAR(128));

An insert would use quotes around the variable name (with a colon in front) since we need to tell Db2 that this is a string.

In [ ]:
%%sql -j
INSERT INTO SHOWJSON VALUES (':employee');
SELECT * FROM SHOWJSON;

An assignment statement to a variable will result in an equivalent Python dictionary type being created. Note that we must use the raw `-j` flag to make sure we only get the data and not a data frame.

In [ ]:
x = %sql -j SELECT * FROM SHOWJSON
print("First Name is " + x[0]["firstname"] + " and the last name is " + x[0]['lastname'])

## Plotting
Sometimes it would be useful to display a result set as either a bar, pie, or line chart. The first one or two
columns of a result set need to contain the values need to plot the information.

The three possible plot options are:
    
* `-pb` - bar chart (x,y)
* `-pp` - pie chart (y)
* `-pl` - line chart (x,y)

The following data will be used to demonstrate the different charting options.

In [ ]:
%sql values 1,2,3,4,5

Since the results only have one column, the pie, line, and bar charts will not have any labels associated with
them. The first example is a bar chart.

In [ ]:
%sql -pb values 1,2,3,4,5

The same data as a pie chart.

In [ ]:
%sql -pp values 1,2,3,4,5

And finally a line chart.

In [ ]:
%sql -pl values 1,2,3,4,5

If you retrieve two columns of information, the first column is used for the labels (X axis or pie slices) and 
the second column contains the data. 

In [ ]:
%sql -pb values ('A',1),('B',2),('C',3),('D',4),('E',5)

For a pie chart, the first column is used to label the slices, while the data comes from the second column.

In [ ]:
%sql -pp values ('A',1),('B',2),('C',3),('D',4),('E',5)

Finally, for a line chart, the x contains the labels and the y values are used.

In [ ]:
%sql -pl values ('A',1),('B',2),('C',3),('D',4),('E',5)

The following SQL will plot the number of employees per department.

In [ ]:
%%sql -pb
SELECT WORKDEPT, COUNT(*) 
  FROM EMPLOYEE
GROUP BY WORKDEPT

The final option for plotting data is to use interactive mode `-i`. This will display the data using an open-source project called Pixiedust. You can view the results in a table and then interactively create a plot by dragging and dropping column names into the appropriate slot. The next command will place you into interactive mode.

In [ ]:
%sql -i select * from employee

## Sample Data
Many of the Db2 notebooks depend on two of the tables that are found in the `SAMPLE` database. Rather than
having to create the entire `SAMPLE` database, this option will create and populate the `EMPLOYEE` and 
`DEPARTMENT` tables in your database. Note that if you already have these tables defined, they will not be dropped.

In [ ]:
%sql -sampledata

## Result Sets 
By default, any `%sql` block will return the contents of a result set as a table that is displayed in the notebook. The results are displayed using a feature of pandas dataframes. The following select statement demonstrates a simple result set.

In [ ]:
%sql select * from employee fetch first 3 rows only

You can assign the result set directly to a variable.

In [ ]:
x = %sql select * from employee fetch first 3 rows only

The variable x contains the dataframe that was produced by the `%sql` statement so you access the result set by using this variable or display the contents by just referring to it in a command line.

In [ ]:
x

There is an additional way of capturing the data through the use of the `-r` flag.
<pre>
var = %sql -r select * from employee
</pre>
Rather than returning a dataframe result set, this option will produce a list of rows which includes the names of the columns in the first row. Each row is a list itself. The column names are found in row zero (0) and the data rows start at 1. To access the first column of the first row, you would use var[1][0] to access it.

In [ ]:
rows = %sql -r select * from employee fetch first 3 rows only
print(rows[1][0])

The number of rows in the result set can be determined by using the length function and subtracting one for the header row.

In [ ]:
print(len(rows)-1)

If you want to iterate over all of the rows and columns, you could use the following Python syntax instead of
creating a for loop that goes from 0 to 41.

In [ ]:
for row in rows:
    line = ""
    for col in row:
        line = line + str(col) + ","
    print(line)

If you don't want the header row, modify the first line to start at the first row instead of row zero.

In [ ]:
for row in rows[1:]:
    line = ""
    for col in row:
        line = line + str(col) + ","
    print(line)

The data is returned as character string, so if you want to do any arithmetic with these columns, you need to convert them.Since the data may be returned in different formats (like integers), you should use the str() function
to convert the values to strings. Otherwise, the concatenation function used in the above example will fail. For
instance, the 6th field is a birthdate field. If you retrieve it as an individual value and try and concatenate a string to it, you get the following error.

In [ ]:
print("Birth Date="+rows[1][8])

You can fix this problem by adding the str function to convert the date.

In [ ]:
print("Birth Date="+str(rows[1][6]))

#### Credits: IBM 2018, George Baklarz [baklarz@ca.ibm.com]